<a href="https://colab.research.google.com/github/invi-bhagyesh/study-GPT/blob/main/training_kr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/invi-bhagyesh/study-GPT.git

Cloning into 'study-GPT'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 109 (delta 28), reused 98 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (109/109), 4.40 MiB | 4.85 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/study-GPT

/content/drive/MyDrive/study-GPT


In [4]:
# Optional: install wandb if you want
# !pip install wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
import math
from tqdm import tqdm

# Optional
# import wandb

In [5]:
from model.gpt2 import GPT2


In [5]:
# For GPT-2
from transformers import GPT2Config
from model.gpt2 import GPT2  # assuming you have your custom GPT2 implementation

# Create config
config = GPT2Config(
    n_layer=6,    # number of transformer layers
    n_head=8,     # number of attention heads
    n_embd=512    # embedding dimension
)

# Initialize model
model = GPT2(config)

# 1. Basic parameter count
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Count parameters
total_params = count_parameters(model)
print(f"Total trainable parameters: {total_params:,}")

Total trainable parameters: 45,171,200


In [6]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [7]:
from transformers import GPT2Tokenizer
from datasets import load_dataset
from train import train_gpt2

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Tokenize
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=1024)

tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

tokenized_dataset.save_to_disk("data/tokenized_wikitext2")

train_dataset = tokenized_dataset["train"]
val_dataset = tokenized_dataset["validation"]



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4358 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/36718 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset["train"]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 36718
})

In [ ]:
total_tokens = sum([len(tokenizer(x['text'])['input_ids']) for x in dataset['train']])
print(total_tokens)

2391884


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from model.config import GPT2Config
from model.gpt2 import GPT2
from transformers import GPT2Tokenizer
from tqdm.notebook import tqdm   # NOTE: notebook-friendly tqdm
import wandb

In [9]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: invi-bhagyesh (invi-bhagyesh-manipal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [10]:
from train import train_gpt2

In [11]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

train_loader = DataLoader(tokenized_dataset["train"], batch_size=8, shuffle=True, collate_fn=data_collator)
val_loader = DataLoader(tokenized_dataset["validation"], batch_size=8, collate_fn=data_collator)


Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [12]:
train_gpt2(train_loader, val_loader, n_epochs=5)
# Save the model after training
torch.save(model.state_dict(), "data/gpt2_pretrained.pth")
print("Model saved to data/gpt2_pretrained_invi.pth")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: invi-bhagyesh (invi-bhagyesh-manipal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/5: 100%|██████████| 4590/4590 [34:45<00:00,  2.20it/s]


Epoch 1: Train Loss = nan, Val Loss = 0.0329
Model saved at ./saved_models/gpt2_epoch_1.pth


Epoch 2/5: 100%|██████████| 4590/4590 [34:55<00:00,  2.19it/s]


Epoch 2: Train Loss = nan, Val Loss = 0.0330
Model saved at ./saved_models/gpt2_epoch_2.pth


Epoch 3/5: 100%|██████████| 4590/4590 [34:53<00:00,  2.19it/s]


Epoch 3: Train Loss = 0.0026, Val Loss = 0.0350
Early stopping triggered!


batch_loss,█▇▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▅█
train_loss,▁
val_loss,▁▁█
batch_loss,0.00092
epoch,2
train_loss,0.00256
val_loss,0.03497


Model saved to data/gpt2_pretrained_invi.pth


In [ ]:
# Train
model = train_gpt2(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    n_epochs=3,
    batch_size=8,
    learning_rate=3e-4
)

In [10]:
import torch
from model.config import GPT2Config
from model.gpt2 import GPT2

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model config
config = GPT2Config()

# Initialize model
model = GPT2(config).to(device)

# Load trained weights
checkpoint_path = "saved_models/gpt2_epoch_2.pth"  # Change to your checkpoint path
model.load_state_dict(torch.load(checkpoint_path, map_location=device), strict=False)
print(f"✅ Model loaded from {checkpoint_path} and ready for inference.")

✅ Model loaded from saved_models/gpt2_epoch_2.pth and ready for inference.


In [13]:
from transformers import GPT2Tokenizer

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Encode input text
input_text = "The future of AI is"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# Generate text
with torch.no_grad():
    output_ids = model.generate(input_ids, max_length=50, num_return_sequences=1)

# Decode generated output
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(f"📝 Generated text: {output_text}")

AttributeError: 'GPT2' object has no attribute 'generate'

In [20]:
import torch
from transformers import GPT2Tokenizer

# Load tokenizer (use same tokenizer used for training)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

def generate_text(model, input_text, max_length=50, device="cuda"):
    model.eval()  # Set to evaluation mode
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Greedy decoding
    with torch.no_grad():
        for _ in range(max_length):
            logits = model(input_ids)  # Forward pass
            next_token = torch.argmax(logits[:, -1, :], dim=-1)  # Get most likely next token
            input_ids = torch.cat([input_ids, next_token.unsqueeze(0)], dim=-1)  # Append to input

            if next_token.item() == tokenizer.eos_token_id:  # Stop if EOS token is generated
                break

    return tokenizer.decode(input_ids.squeeze(), skip_special_tokens=True)

# Load your model
from model.config import GPT2Config
from model.gpt2 import GPT2  # Ensure correct import
config = GPT2Config()  # Load same config used during training
model = GPT2(config)
model.load_state_dict(torch.load("saved_models/gpt2_epoch_2.pth", map_location="cuda"))
model.to("cuda")

# **Run Inference**
input_text = "Once upon a time"
output_text = generate_text(model, input_text)
print(output_text)

RuntimeError: Error(s) in loading state_dict for GPT2:
	Unexpected key(s) in state_dict: "blocks.6.ln_1.weight", "blocks.6.ln_1.bias", "blocks.6.attn.bias", "blocks.6.attn.c_attn.weight", "blocks.6.attn.c_attn.bias", "blocks.6.attn.c_proj.weight", "blocks.6.attn.c_proj.bias", "blocks.6.ln_2.weight", "blocks.6.ln_2.bias", "blocks.6.mlp.0.weight", "blocks.6.mlp.0.bias", "blocks.6.mlp.2.weight", "blocks.6.mlp.2.bias", "blocks.7.ln_1.weight", "blocks.7.ln_1.bias", "blocks.7.attn.bias", "blocks.7.attn.c_attn.weight", "blocks.7.attn.c_attn.bias", "blocks.7.attn.c_proj.weight", "blocks.7.attn.c_proj.bias", "blocks.7.ln_2.weight", "blocks.7.ln_2.bias", "blocks.7.mlp.0.weight", "blocks.7.mlp.0.bias", "blocks.7.mlp.2.weight", "blocks.7.mlp.2.bias". 